In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!python -m pip install pyyaml
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Cloning into 'detectron2'...
remote: Enumerating objects: 15773, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 15773 (delta 6), reused 26 (delta 4), pack-reused 15743 (from 1)
Receiving objects: 100% (15773/15773), 6.35 MiB | 19.63 MiB/s, done.
Resolving deltas: 100% (11502/11502), done.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.9 MB/s eta 0:00:

In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.4 ; cuda:  cu121
detectron2: 0.6


In [4]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
data_dir_path = "/content/drive/MyDrive/instseg/data/"

In [6]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("taco_train", {}, data_dir_path + "mapped_annotations_0_train.json", data_dir_path + "images/")
register_coco_instances("taco_val", {}, data_dir_path + "mapped_annotations_0_val.json", data_dir_path + "images/")
register_coco_instances("taco_test", {}, data_dir_path + "mapped_annotations_0_test.json", data_dir_path + "images/")

In [7]:
import json

# Load your JSON file
with open('/content/drive/MyDrive/instseg/data/mapped_annotations_0_train.json') as f:
    data = json.load(f)

# Generate unique IDs
unique_id = 0
for annotation in data['annotations']:
    annotation['id'] = unique_id
    unique_id += 1

# Save the corrected JSON back to file
with open('/content/drive/MyDrive/instseg/data/mapped_annotations_0_train.json', 'w') as f:
    json.dump(data, f, indent=4)


In [8]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [9]:
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer

In [10]:
from detectron2.data import build_detection_train_loader
class Trainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    return COCOEvaluator(dataset_name, output_dir=output_folder)

In [11]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("taco_train",)
cfg.DATASETS.TEST = ("taco_val",)
cfg.TEST.EVAL_PERIOD = 6000
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 60000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[09/02 06:49:58 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 135MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[09/02 06:49:59 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/02 06:50:38 d2.utils.events]:  eta: 1 day, 5:39:17  iter: 19  total_loss: 3.597  loss_cls: 2.131  loss_box_reg: 0.6444  loss_mask: 0.6901  loss_rpn_cls: 0.08489  loss_rpn_loc: 0.02437    time: 1.6771  last_time: 0.2740  data_time: 1.4043  last_data_time: 0.0012   lr: 1.9981e-05  max_mem: 2183M
[09/02 06:51:06 d2.utils.events]:  eta: 1 day, 0:05:39  iter: 39  total_loss: 3.182  loss_cls: 1.701  loss_box_reg: 0.6469  loss_mask: 0.6805  loss_rpn_cls: 0.1124  loss_rpn_loc: 0.0346    time: 1.4533  last_time: 1.7041  data_time: 0.9656  last_data_time: 1.4510   lr: 3.9961e-05  max_mem: 2183M
[09/02 06:51:35 d2.utils.events]:  eta: 1 day, 0:32:06  iter: 59  total_loss: 2.416  loss_cls: 1.017  loss_box_reg: 0.5403  loss_mask: 0.6658  loss_rpn_cls: 0.0665  loss_rpn_loc: 0.01335    time: 1.4479  last_time: 1.5161  data_time: 1.1641  last_data_time: 1.2791   lr: 5.9941e-05  max_mem: 2263M
[09/02 06:52:04 d2.utils.events]:  eta: 1 day, 0:31:36  iter: 79  total_loss: 1.949  loss_cls: 0.6574  los

...

[09/02 12:24:18 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0014 s/iter. Inference: 0.0794 s/iter. Eval: 0.1350 s/iter. Total: 0.2158 s/iter. ETA=0:00:29
[09/02 12:24:24 d2.evaluation.evaluator]: Inference done 30/150. Dataloading: 0.0405 s/iter. Inference: 0.0786 s/iter. Eval: 0.1393 s/iter. Total: 0.2586 s/iter. ETA=0:00:31
[09/02 12:24:29 d2.evaluation.evaluator]: Inference done 41/150. Dataloading: 0.0407 s/iter. Inference: 0.0806 s/iter. Eval: 0.2033 s/iter. Total: 0.3247 s/iter. ETA=0:00:35
[09/02 12:24:34 d2.evaluation.evaluator]: Inference done 60/150. Dataloading: 0.0458 s/iter. Inference: 0.0806 s/iter. Eval: 0.1834 s/iter. Total: 0.3100 s/iter. ETA=0:00:27
[09/02 12:24:39 d2.evaluation.evaluator]: Inference done 80/150. Dataloading: 0.0351 s/iter. Inference: 0.0810 s/iter. Eval: 0.1784 s/iter. Total: 0.2946 s/iter. ETA=0:00:20
[09/02 12:24:44 d2.evaluation.evaluator]: Inference done 110/150. Dataloading: 0.0307 s/iter. Inference: 0.0792 s/iter. Eval: 0.14

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[09/02 13:04:43 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0048 s/iter. Inference: 0.0829 s/iter. Eval: 0.3089 s/iter. Total: 0.3967 s/iter. ETA=0:00:55
[09/02 13:04:49 d2.evaluation.evaluator]: Inference done 27/150. Dataloading: 0.0327 s/iter. Inference: 0.0806 s/iter. Eval: 0.2374 s/iter. Total: 0.3509 s/iter. ETA=0:00:43
[09/02 13:04:55 d2.evaluation.evaluator]: Inference done 41/150. Dataloading: 0.0444 s/iter. Inference: 0.0822 s/iter. Eval: 0.2634 s/iter. Total: 0.3901 s/iter. ETA=0:00:42
[09/02 13:05:00 d2.evaluation.evaluator]: Inference done 59/150. Dataloading: 0.0370 s/iter. Inference: 0.0827 s/iter. Eval: 0.2358 s/iter. Total: 0.3558 s/iter. ETA=0:00:32
[09/02 13:05:05 d2.evaluation.evaluator]: Inference done 75/150. Dataloading: 0.0331 s/iter. Inference: 0.0833 s/iter. Eval: 0.2294 s/iter. Total: 0.3460 s/iter. ETA=0:00:25
[09/02 13:05:10 d2.evaluation.evaluator]: Inference done 93/150. Dataloading: 0.0282 s/iter. Inference: 0.0824 s/iter. Eval: 0.221

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[09/02 13:45:13 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0043 s/iter. Inference: 0.0822 s/iter. Eval: 0.1704 s/iter. Total: 0.2569 s/iter. ETA=0:00:35
[09/02 13:45:18 d2.evaluation.evaluator]: Inference done 31/150. Dataloading: 0.0450 s/iter. Inference: 0.0789 s/iter. Eval: 0.1302 s/iter. Total: 0.2543 s/iter. ETA=0:00:30
[09/02 13:45:23 d2.evaluation.evaluator]: Inference done 41/150. Dataloading: 0.0537 s/iter. Inference: 0.0810 s/iter. Eval: 0.1883 s/iter. Total: 0.3232 s/iter. ETA=0:00:35
[09/02 13:45:28 d2.evaluation.evaluator]: Inference done 57/150. Dataloading: 0.0471 s/iter. Inference: 0.0814 s/iter. Eval: 0.1924 s/iter. Total: 0.3210 s/iter. ETA=0:00:29
[09/02 13:45:34 d2.evaluation.evaluator]: Inference done 62/150. Dataloading: 0.0465 s/iter. Inference: 0.0825 s/iter. Eval: 0.2603 s/iter. Total: 0.3895 s/iter. ETA=0:00:34
[09/02 13:45:39 d2.evaluation.evaluator]: Inference done 74/150. Dataloading: 0.0388 s/iter. Inference: 0.0829 s/iter. Eval: 0.273

## Evaluation on TACO10 Test Split

In [14]:
from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader

cfg.DATASETS.TEST = ("taco_test",)
evaluator = COCOEvaluator("taco_test", output_dir="output")
test_loader = build_detection_test_loader(cfg, "taco_test")
metrics = inference_on_dataset(trainer.model, test_loader, evaluator)
print(metrics)

[09/02 13:46:49 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
[09/02 13:46:50 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/drive/MyDrive/instseg/data/mapped_annotations_0_test.json
[09/02 13:46:50 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|      Can      | 37           |   Other    | 151          |   Bottle   | 54           |
|  Bottle cap   | 41           |    Cup     | 17           |    Lid     | 15           |
| Plastic bag.. | 84           |  Pop tab   | 11           |   Straw    | 38           |
|   Cigarette   | 121          |            |              |            |              |
|     total     | 569          |            |              |            |              |
[09/02 13:46:50 d2.data.dataset_mapp

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[09/02 13:46:58 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.4796 s/iter. Inference: 0.0762 s/iter. Eval: 0.1103 s/iter. Total: 0.6660 s/iter. ETA=0:01:32
[09/02 13:47:05 d2.evaluation.evaluator]: Inference done 21/150. Dataloading: 0.5044 s/iter. Inference: 0.0748 s/iter. Eval: 0.0889 s/iter. Total: 0.6683 s/iter. ETA=0:01:26
[09/02 13:47:11 d2.evaluation.evaluator]: Inference done 29/150. Dataloading: 0.5338 s/iter. Inference: 0.0769 s/iter. Eval: 0.1080 s/iter. Total: 0.7189 s/iter. ETA=0:01:26
[09/02 13:47:22 d2.evaluation.evaluator]: Inference done 37/150. Dataloading: 0.5224 s/iter. Inference: 0.0839 s/iter. Eval: 0.2657 s/iter. Total: 0.8722 s/iter. ETA=0:01:38
[09/02 13:47:28 d2.evaluation.evaluator]: Inference done 39/150. Dataloading: 0.4918 s/iter. Inference: 0.0869 s/iter. Eval: 0.4300 s/iter. Total: 1.0089 s/iter. ETA=0:01:51
[09/02 13:47:35 d2.evaluation.evaluator]: Inference done 48/150. Dataloading: 0.4372 s/iter. Inference: 0.0860 s/iter. Eval: 0.417